# Mise en place de l'environnement

In [1]:
# On importe les packages nécesaires au machine learning
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, recall_score,precision_score, roc_auc_score  

In [2]:
# On met en place Mlflow pour suivre les performances de nos modèles
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse

# On crée une nouvelle expérimentation qui permet de rassembler tous les entrainements pour un meme projet.
# experiment_id = mlflow.create_experiment("classification_tutorial")

# Data Preparation

In [9]:
# On importe la table nettoyée et augmentée des nouveaux attributs.
df = pd.read_csv("data/intermediate/Telco_post_analysis.csv")

## Encodage

La pluspart des modèles de machine learning n'acceptent pas les vairables catégorielles (string). Il faut donc passer par une première étape de les transformer en variable numérique. Cette opération est appellée encodage. Voici les trois encodages principaux dans la librairie scikit learn:
- [sklearn.preprocessing.LabelEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html) : transforme la variable catégorielle cible en variable numérique. (chaque classe sera représentée par un chiffre)
- [sklearn.preprocessing.OrdinalEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html#sklearn.preprocessing.OrdinalEncoder) Transforme une variable explicative catégorielle en variable numérique, chaque classe correspond à un chiffre. L'ordre des chiffres est supposé avoir un sens. On le privilègie donc pour les variables catégorielles dites "ordinales" du types: satifaction (low, medium, high)
- [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) Transforme une variable explicative catégorielle en autant de variable binaire (0/1) que la variable catégorielle à de valeur (-1).
    - exemple: la variable couleur_cheveux dont les valeurs sont (brun, blond, roux, chauve) sera transformer en trois variables binaires: couleur_cheveux_brun (0/1), couleur_cheveux_blond (0/1), couleur_cheveux_roux (0/1). Si une personne est chauve elle aura 0 aux trois variables précédentes, pas besoin d'ajouter donc cette varaible supplémentaire.

OneHotEncoder est pratique à utiliser dans un pipe scikit learn mais il peut être moins pratique à utiliser avec un dataframe pandas. Pour hot encoder un dataframe on peut utiliser [pandas.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html)

### Encodage du Churn avec Label encoder

In [ ]:
df["Churn"].describe

In [11]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df["Churn"]= le.fit_transform(df["Churn"])

In [26]:
df["Churn"].describe

<bound method NDFrame.describe of 0        No
1        No
2       Yes
3        No
4       Yes
       ... 
7027     No
7028     No
7029     No
7030    Yes
7031     No
Name: Churn, Length: 7032, dtype: object>

### Encodage des variables explicatives avec get_dummies

In [30]:
varlist =  ['gender','SeniorCitizen','Partner', 'Dependents','PhoneService', 'MultipleLines',
            'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
            'StreamingTV', 'StreamingMovies', 'Contract','PaperlessBilling', 'PaymentMethod' ,'Service' 
            ]

df_encode = pd.get_dummies(df[varlist], drop_first=True) #drop first permet de oneHotencoder plutot que de get_dummies
df_encode = df.drop(varlist, axis=1).join(df_encode)
df_encode.head()

,customerID,tenure,MonthlyCharges,TotalCharges,Churn,nbr_option_internet,SeniorCitizen,gender_Male,Partner_Yes,Dependents_Yes,...,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Service_Only Phone,Service_Phone and Internet
0,7590-VHVEG,1,29.85,29.85,No,1,0,0,1,0,...,0,0,0,0,1,0,1,0,0,0
1,5575-GNVDE,34,56.95,1889.50,No,2,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
2,3668-QPYBK,2,53.85,108.15,Yes,2,0,1,0,0,...,0,0,0,0,1,0,0,1,0,1
3,7795-CFOCW,45,42.30,1840.75,No,3,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,9237-HQITU,2,70.70,151.65,Yes,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1


L'encodage est aussi le moment pour selectionner les variables qui nous interesse le plus.
On a par exemple:
- Internet : Oui, non
- option internet: oui, non, pas internet

les indicatrices (dummies) Internet_non et option_internet_pas_internet sont exactement les memes, on peut donc en concerver qu'une sur deux

### example d'encodage ordinal avec apply map

In [5]:
# Defining the map function
def binary_map(x):
    return x.map({'low': 0, "medium": 1,"high": 2})

# Applying the function to the housing list
#df.quality = df.quality.apply(binary_map)

## Separation de la DB

In [6]:
# On divise la base en train et test
y = df["Churn"]
X = df[["SeniorCitizen","nbr_option_internet","tenure"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 1)


 # Choix de la metrique d'évalulation, du modèle et la gestion du déséquilibre du jeu de données

A propos des jeux de données non équilibrées: [this link](https://elitedatascience.com/imbalanced-classes)

# Premiere regression avec MLflow et Scikitlearn

In [7]:
def eval_metrics(actual, pred):
    accuracy = accuracy_score(actual, pred)
    f1 = f1_score(actual, pred)
    recall = recall_score(actual, pred)
    precision = precision_score(actual, pred)
    auc = roc_auc_score(actual, pred)
    return accuracy, f1, recall, precision, auc


In [9]:
# On utilise la fonction autolog qui permet de tracker tous les paramètres des modèles
mlflow.sklearn.autolog()

with mlflow.start_run(experiment_id = 1):
    lg = LogisticRegression()
    model = lg.fit(X_train,y_train)
    y_pred = model.predict(X_test)

    accuracy, f1, recall, precision, auc = eval_metrics(y_test, y_pred)
    print("  accuracy: %s" % accuracy)
    print("  f1: %s" % f1)
    print("  recall: %s" % recall) 
    print("  precision: %s" % precision) 
    print("  auc: %s" % auc) 

    #mlflow.log_param("n_estimators", n_estimators)
    # mlflow.log_metric("accuracy", accuracy)
    # mlflow.log_metric("f1", f1)
    # mlflow.log_metric("recall", recall)
    # mlflow.log_metric("precision", precision)
    # mlflow.log_metric("auc", auc)                 

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store != "file":
        mlflow.sklearn.log_model(lg, "model", registered_model_name="simple_logisticreggression")
    else:
        mlflow.sklearn.log_model(lg, "model") 

2022/02/19 18:36:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/charles/Documents/pythonProject/Introduction_ml_classification/env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_confusion_matrix is deprecated; Function `plot_confusion_matrix` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: ConfusionMatrixDisplay.from_predictions or ConfusionMatrixDisplay.from_estimator."
2022/02/19 18:36:41 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/charles/Documents/pythonProject/Introduction_ml_classification/env/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_e

  accuracy: 0.746268656716418
  f1: 0.3376623376623376
  recall: 0.2345360824742268
  precision: 0.6026490066225165
  auc: 0.5878274131703813


# Première regression avec Stat models

In [10]:
import statsmodels.api as sm
import mlflow.statsmodels

In [26]:
mlflow.statsmodels.autolog()

with mlflow.start_run(experiment_id = 1):
    # Logistic regression model
    log_reg = sm.Logit(y_train, X_train).fit()
    yhat = log_reg.predict(X_test)
    y_pred = list(map(round, yhat))


    accuracy, f1, recall, precision, auc = eval_metrics(y_test, y_pred)
    print("  accuracy: %s" % accuracy)
    print("  f1: %s" % f1)
    print("  recall: %s" % recall) 
    print("  precision: %s" % precision) 
    print("  auc: %s" % auc) 

    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("auc", auc) 
    

2022/02/19 21:55:33 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of statsmodels. If you encounter errors during autologging, try upgrading / downgrading statsmodels to a supported version, or try upgrading MLflow.


Optimization terminated successfully.
         Current function value: 0.486836
         Iterations 6
  accuracy: 0.757640369580668
  f1: 0.44006568144499175
  recall: 0.34536082474226804
  precision: 0.6063348416289592
  auc: 0.6299915016743726


# Consignes

Essayer plusieurs regressions logistiques en faisant varier les variables en input (pensez à tester une discrétisation des varibles continues) et les paramètres afin d'optimiser la métrique prise pour cible.